In [1]:
import tensorflow as tf 
from tensorflow import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import *
from keras.optimizers import *
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

In [2]:
# Load the dataset

(x_train,y_train) , (x_test,y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


In [3]:
# Preprocess the images
x_train = x_train.astype('float32') / 255.0
x_test= x_test.astype('float32') / 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



In [4]:
# Define the VGG16 model with the last layer removed
vgg16 = VGG16(include_top=False,input_shape=(32,32,3))

58889256/58889256 [==============================] - 2s 0us/step


In [5]:
from keras.layers.attention.multi_head_attention import activation
flatten = Flatten()(vgg16.output)
dense1 = Dense(4096,activation='relu')(flatten)
dense2 = Dense(4096,activation='relu')(dense1)
output = Dense(10,activation='softmax')(dense2)


In [6]:
model = Model(inputs=vgg16.input,outputs=output)

In [7]:
for layer in vgg16.layers:
  layer.trainable = False

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

# model compile

In [9]:
model.compile(optimizer=SGD(learning_rate=0.001),loss='categorical_crossentropy',metrics=['acc'])

# Data augmentation

In [10]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=[0.9,1.1],
    fill_mode='constant',
    cval=0.0
)

# Train the model

In [11]:
history = model.fit(datagen.flow(x_train,y_train,batch_size=128),epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
391/391 [==============================] - 43s 87ms/step - loss: 2.2495 - acc: 0.2223 - val_loss: 2.2121 - val_acc: 0.2654
Epoch 2/100
391/391 [==============================] - 33s 84ms/step - loss: 2.1648 - acc: 0.3062 - val_loss: 2.1447 - val_acc: 0.2926
Epoch 3/100
391/391 [==============================] - 34s 88ms/step - loss: 2.0962 - acc: 0.3268 - val_loss: 2.0921 - val_acc: 0.2980
Epoch 4/100
391/391 [==============================] - 33s 84ms/step - loss: 2.0397 - acc: 0.3384 - val_loss: 2.0460 - val_acc: 0.3092
Epoch 5/100
391/391 [==============================] - 32s 82ms/step - loss: 1.9906 - acc: 0.3457 - val_loss: 2.0065 - val_acc: 0.3194
Epoch 6/100
391/391 [==============================] - 34s 86ms/step - loss: 1.9475 - acc: 0.3538 - val_loss: 1.9712 - val_acc: 0.3260
Epoch 7/100
391/391 [==============================] - 33s 83ms/step - loss: 1.9112 - acc: 0.3598 - val_loss: 1.9439 - val_acc: 0.3286
Epoch 8/100
391/391 [==============================] - 

In [12]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

313/313 [==============================] - 3s 8ms/step - loss: 1.5275 - acc: 0.4540
Test accuracy: 0.45399999618530273
